In [1]:
import os
import warnings
import radiomics 
import numpy as np
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt
from nilearn import plotting, image
from matplotlib.colors import ListedColormap

/tmp/ipykernel_36142/2318182290.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Path directories

In [2]:
# Main path where all requiered files are encountered.
mainPath = os.path.join('/home/danvalcor/Documents/ANDI/Sample010')

# Specific path for the volumes.
asegPath = f'{mainPath}/mri/aparc+aseg.mgz'
brainPath = f'{mainPath}/mri/brain.mgz'

# We are also including the stats file to save the known stats.
asegStats = f"{mainPath}/stats/aseg.stats"

# Path to load the LUT table
LUTfile = os.path.join(os.environ['FREESURFER_HOME'],'luts/FreeSurferColorLUT.txt')

# Load the MRI libraries using the nibabel library
brain = nib.load(brainPath)   # Volume containing the MRI of the brain.
aseg = nib.load(asegPath)     # Volume containing the segmentations of the brains MRI

### LUT load

In [3]:
# Create a data Frame to contain all relevant information of the LUT
df = pd.DataFrame(columns=["Id", "Segmentation","Color Array"])

# Declares the path where the LUT file is stored inside FreeSurfers Directory. 
lut_data = []

# Opens and reads the file
with open(LUTfile, "r") as file:
    content = file.readlines()

# Reads each line of the file
for line in content:
    # Skip empty lines
    if len(line) > 1:
        # Strip, split and join each line by the tab ('\t') separator, to assign each value easier
        sLine = line.strip().split("\t")
        sLine = " ".join(sLine).split()
        # Checks and skipps anotations in the file
        if '#' not in sLine[0]:
            # Separates and assigns the contained values to the dictionary
            labelId = int(sLine[0])                     # Converts the numbers back into integers.
            labelName = sLine[1]                        # Assigns the structure labels.
            rgbColors = [int(p) for p in sLine[-4:-1]]  # Converts each rgb value back into integers.
            lut_data.append({"Id": labelId, "StructName": labelName, "Color Array": rgbColors})

# Create DataFrame from the list of dictionaries
df = pd.DataFrame(lut_data)# Prints the dictionary to verify


### Color Map

In [4]:
# Create a dictionary mapping the ID to its corresponding RGB color
id_to_color = dict(zip(df['Id'], df['Color Array']))

# Generate an array of RGB colors for each ID
colors_rgb = [id_to_color[id_] for id_ in df['Id']]

# Create the colormap with the RGB colors
cmap = ListedColormap(colors_rgb)

### Segment filtering

In [5]:
# We obtain and save the data present in our segmentation volume.
img_data = aseg.get_fdata()

# Creates a temporary array to store the number of voxels present in every segmet.
calculated = []

# iterates each row present in the Data Frame
for i, row in df.iterrows():
    # Assigns the id, structure Name, and corresponding color
    colorId, name, color = row[['Id','StructName','Color Array']]
    # Create a boolean mask for values in img_data matching the Id value
    mask = (img_data == colorId)
    # Calculate the number of voxels
    num_voxels = np.count_nonzero(mask)
    calculated.append(num_voxels)

# Add the calculated list as a new column named "calculated" to the DataFrame
df['NVoxels'] = calculated
# Remove rows where 'NVoxels' is 0, and resets the index of the dataFrame
df = df[df['NVoxels'] != 0].reset_index(drop=True)

# Show the resulting DataFrame
print("Present Segments: ",len(df))
print(df)

Present Segments:  111
       Id                    StructName      Color Array   NVoxels
0       0                       Unknown        [0, 0, 0]  15611097
1       2    Left-Cerebral-White-Matter  [245, 245, 245]    237966
2       4        Left-Lateral-Ventricle   [120, 18, 134]     22632
3       5             Left-Inf-Lat-Vent   [196, 58, 250]      1001
4       7  Left-Cerebellum-White-Matter  [220, 248, 164]     14400
..    ...                           ...              ...       ...
106  2031          ctx-rh-supramarginal    [80, 160, 20]      8190
107  2032            ctx-rh-frontalpole    [100, 0, 100]       921
108  2033           ctx-rh-temporalpole     [70, 70, 70]      2314
109  2034     ctx-rh-transversetemporal  [150, 150, 200]       665
110  2035                 ctx-rh-insula   [255, 192, 32]      6983

[111 rows x 4 columns]


### PyRadiomics Characteristics

In [12]:
# Create a SimpleITK image from a numpy array of aseg data with int32 data type
brain_sitk = sitk.GetImageFromArray(np.array(aseg.dataobj, dtype=np.int32))

# Crear una instancia del extractor de características
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor()

# Configurar las opciones del extractor
extractor.settings['shape'] = True  # Habilitar características de forma
extractor.settings['firstorder'] = True  # Habilitar características de primer orden
extractor.settings['glcm'] = False  # Deshabilitar características de matriz de co-ocurrencia de niveles de gris
extractor.settings['glrlm'] = False  # Deshabilitar características de matriz de longitud de corrida de niveles de gris
extractor.settings['glszm'] = False  # Deshabilitar características de matriz de tamaño de zona de niveles de gris
extractor.settings['ngtdm'] = False  # Deshabilitar características de matriz de diferencia de textura de nivel de gris
# Puedes configurar otras opciones según tus necesidades

#### Specific Characteristics

In [13]:
# Create a list to store the feature results
results = []
# Desactivar las advertencias de tipo UserWarning temporalmente (Me desespere)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # Iterate over rows in the DataFrame
    for i, row in df.iterrows():
        # Check if it's not the first row and the NVoxels value is greater than 1
        if i > 0 and row['NVoxels'] > 1:
            # Extract ColorId, StructName, and Color from the current row
            colorId, name, color = row[['Id', 'StructName', 'Color Array']]
            
            # Create a boolean mask for img_data values that match the ColorId value
            mask = (img_data == colorId)
            
            # Print progress approximately every 5%
            if i / len(df) * 100 % 5 < 1:  # Check if the remainder of division is approximately 0
                print(f"Progress: {i / len(df) * 100:.0f}%")
            
            # Apply the logical mask
            masked_img_data = np.where(mask, colorId, 0)
            masked_img = image.new_img_like(aseg, masked_img_data.astype(np.int32))

            # Convert Nilearn images to SimpleITK
            segmentation_sitk = sitk.GetImageFromArray(np.array(masked_img.dataobj, dtype=np.int32))
            
            # Extract features
            features = extractor.execute(brain_sitk,segmentation_sitk,  colorId)

            # Add the feature results to the DataFrame
            results.append({'ColorId': colorId, 'StructName': name, **{feature_name: features[feature_name] for feature_name in features.keys()}})

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results)

# Print the DataFrame with the results
print(results_df)    

Progress: 1%


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Progress: 5%


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Progress: 11%


...

Progress: 91%


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Progress: 95%


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


     ColorId                    StructName diagnostics_Versions_PyRadiomics  \
0          2    Left-Cerebral-White-Matter                           v3.0.1   
1          4        Left-Lateral-Ventricle                           v3.0.1   
2          5             Left-Inf-Lat-Vent                           v3.0.1   
3          7  Left-Cerebellum-White-Matter                           v3.0.1   
4          8        Left-Cerebellum-Cortex                           v3.0.1   
..       ...                           ...                              ...   
104     2031          ctx-rh-supramarginal                           v3.0.1   
105     2032            ctx-rh-frontalpole                           v3.0.1   
106     2033           ctx-rh-temporalpole                           v3.0.1   
107     2034     ctx-rh-transversetemporal                           v3.0.1   
108     2035                 ctx-rh-insula                           v3.0.1   

    diagnostics_Versions_Numpy diagnostics_Versions

In [9]:
print(results_df.iloc[1])

ColorId                                                  4
StructName                          Left-Lateral-Ventricle
diagnostics_Versions_PyRadiomics                    v3.0.1
diagnostics_Versions_Numpy                          1.26.4
diagnostics_Versions_SimpleITK                       2.3.1
                                             ...          
original_ngtdm_Busyness                                0.0
original_ngtdm_Coarseness                        1000000.0
original_ngtdm_Complexity                              0.0
original_ngtdm_Contrast                                0.0
original_ngtdm_Strength                                0.0
Name: 1, Length: 131, dtype: object
